You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer 

Final Answer: 

Content Plan Document

Title: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of the importance of Artificial Intelligence (AI) in various industries
- Mention of the rapid advancements and innovations in AI technology

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural language processing and sentiment analysis
- AI-powered automation and robotics
- AI ethics and bias mitigation

2. Key Players in t

I now can give a great answer

Final Answer:
# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a driving force in various industries, revolutionizing processes and capabilities. With rapid advancements and innovations in AI technology, the landscape of business and society is continuously evolving. From machine learning algorithms to natural language processing, AI is reshaping how we interact with technology and information.

## Latest Trends in Artificial Intelligence
One of the prominent trends in AI is the development of machine learning and deep learning algorithms. These technologies enable computers to learn from data and make intelligent decisions without explicit programming. Natural language processing and sentiment analysis are also gaining traction, allowing machines to understand and interpret human language more effectively. AI-powered automation and robotics are transforming industries by streamlining 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a driving force in various industries, revolutionizing processes and capabilities. With rapid advancements and innovations in AI technology, the landscape of business and society is continuously evolving. From machine learning algorithms to natural language processing, AI is reshaping how we interact with technology and information.

## Latest Trends in Artificial Intelligence
One of the prominent trends in AI is the development of machine learning and deep learning algorithms. These technologies enable computers to learn from data and make intelligent decisions without explicit programming. Natural language processing and sentiment analysis are also gaining traction, allowing machines to understand and interpret human language more effectively. AI-powered automation and robotics are transforming industries by streamlining operations and enhancing productivity. However, with these advancements come concerns about AI ethics and bias mitigation, emphasizing the importance of responsible AI development.

## Key Players in the AI Industry
Several companies are at the forefront of AI innovation, with Google AI, IBM Watson, Microsoft AI, Amazon AI, Tesla AI, and OpenAI leading the way. These key players have made significant contributions to AI research and applications, shaping the future of technology and its impact on society. Their advancements in machine learning, natural language processing, and robotics have set new standards for AI capabilities and possibilities.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI research have led to exciting applications in healthcare, finance, and marketing. AI is revolutionizing patient care, financial analysis, and customer insights, driving efficiency and accuracy in decision-making. However, ethical concerns surrounding AI development and regulatory developments are critical considerations in ensuring the responsible use of AI technology. As AI continues to evolve, it is essential to address these challenges proactively to harness its full potential for the benefit of society.

In conclusion, Artificial Intelligence is a dynamic field with limitless possibilities and challenges. By staying informed about the latest trends, key players, and noteworthy news in AI, individuals can navigate the complexities of this technology-driven era and contribute to its ethical and sustainable development. Embracing AI with a mindful approach will pave the way for a future where innovation and responsibility coexist harmoniously.

Don't miss out on the latest updates and in-depth articles on Artificial Intelligence. Subscribe to our blog for regular insights and analysis on AI trends and developments. Explore the possibilities of AI with us and join the conversation on shaping the future of technology together.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Synthetic Data to improve defect detection for manufacturing industry"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Synthetic Data to improve defect detection for manufacturing industry.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Leveraging Synthetic Data for Enhanced Defect Detection in the Manufacturing Industry

Introduction:
- Brief explanation of synthetic data and its application in defect detection for the manufacturing industry
- Importance of accurate defect detection in manufacturing processes

Key Points:
1. Latest Trends in Synthetic Data for Defect Detection:
- Use of machine learning algorithms to generate synthetic data for training defect dete

I now can give a great answer

Final Answer: 

# Leveraging Synthetic Data for Enhanced Defect Detection in the Manufacturing Industry

**Introduction:**
In the fast-paced world of manufacturing, accurate defect detection is crucial for ensuring product quality and customer satisfaction. With the rise of synthetic data, generated using machine learning algorithms, companies now have a powerful tool at their disposal to improve defect detection processes. By integrating synthetic data with real-world data, manufacturing companies can enhance the performance of their defect detection models and streamline quality control processes.

**Latest Trends in Synthetic Data for Defect Detection:**
The use of machine learning algorithms to generate synthetic data has become a popular trend in defect detection for the manufacturing industry. This approach allows for the creation of diverse and realistic data sets, which are essential for training robust defect detection models. By combining synthe

In [16]:
Markdown(result)

# Leveraging Synthetic Data for Enhanced Defect Detection in the Manufacturing Industry

**Introduction:**
In the fast-paced world of manufacturing, accurate defect detection is crucial for ensuring product quality and customer satisfaction. With the rise of synthetic data, generated using machine learning algorithms, companies now have a powerful tool at their disposal to improve defect detection processes. By integrating synthetic data with real-world data, manufacturing companies can enhance the performance of their defect detection models and streamline quality control processes.

**Latest Trends in Synthetic Data for Defect Detection:**
The use of machine learning algorithms to generate synthetic data has become a popular trend in defect detection for the manufacturing industry. This approach allows for the creation of diverse and realistic data sets, which are essential for training robust defect detection models. By combining synthetic data with real-world data, companies can improve the accuracy and efficiency of their defect detection systems. Leading manufacturing companies are already adopting synthetic data to enhance their quality control processes, highlighting the importance of this emerging trend in the industry.

**Key Players in Synthetic Data for Defect Detection:**
Several top companies specialize in synthetic data generation for defect detection in the manufacturing industry. These companies offer cutting-edge solutions for creating synthetic data sets that can be used to train advanced defect detection models. Case studies showcasing the successful implementation of synthetic data in defect detection systems further demonstrate the effectiveness of this approach. By partnering with these key players, manufacturing companies can stay ahead of the curve and ensure the quality of their products.

**Noteworthy News on Synthetic Data in Manufacturing:**
Recent advancements in synthetic data technology have paved the way for more efficient defect detection processes in the manufacturing industry. Industry events and collaborations focusing on the use of synthetic data in quality control highlight the growing importance of this technology. By staying informed about the latest trends and developments in synthetic data, manufacturing professionals can make informed decisions about implementing this innovative approach in their defect detection processes.

**Target Audience Analysis:**
Manufacturing professionals, quality control specialists, data scientists, AI engineers, and decision-makers in manufacturing companies can all benefit from leveraging synthetic data for defect detection. By understanding the potential of synthetic data in enhancing quality control processes, industry professionals can make informed decisions about incorporating this technology into their operations. As the manufacturing industry continues to evolve, staying informed about the latest trends in synthetic data is essential for maintaining a competitive edge.

**Conclusion:**
In conclusion, synthetic data has the potential to revolutionize defect detection in the manufacturing industry. By leveraging machine learning algorithms and integrating synthetic data with real-world data, companies can enhance the accuracy and efficiency of their defect detection processes. With the support of top companies specializing in synthetic data generation and the latest advancements in technology, manufacturing professionals have the tools they need to improve quality control and ensure the highest standards for their products. By staying informed about the latest trends and developments in synthetic data, industry professionals can take their defect detection processes to the next level and drive innovation in the manufacturing industry.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).